In [ ]:
import json
import torch
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader, random_split
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader as PyGDataLoader
import torch.nn as nn
from torch_geometric.nn import GCNConv, global_mean_pool
import torch.nn.functional as F
import matplotlib.pyplot as plt
import sys
from torch_geometric.utils import to_dense_batch
import torch.optim as optim
import json

sys.path.append(os.path.abspath(".."))

## Phase 1: Generating the Graphs

#### 1. Download Data

- To download only one episode, use data_downloader(ep_num,...)

- To download a range of episodes, use data_downloader(min_ep,max_ep,...)

- To download a list of episodes, use data_downloader(list,...)

- Additionally, you can chnge the following directories:

    - tabular_data_dir = '../data/raw/L2D/tabular'

    - frames_dir = '../data/raw/L2D/frames'

NOTE: All cameras are currently set to false until we figure out how to match camera data to tabular data in the new release.

In [ ]:
from functions.load_data_L2D import data_downloader

data_downloader(0,n_secs=3,
                features={"tabular": True,
                          "frames": {
                                'observation.images.front_left': False,
                                'observation.images.left_backward': False,
                                'observation.images.left_forward': False,
                                'observation.images.map': False,
                                'observation.images.rear': False,
                                'observation.images.right_backward': False,
                                'observation.images.right_forward': False,
                            }
                        })

#### 2. Process Tabular Data & Add Tags

- To process only one episode, use process_tabular_data(ep_num,...)

- To process a range of episodes, use process_tabular_data(min_ep,max_ep,...)

- To process a list of episodes, use process_tabular_data(list,...)

- Additionally, you can change the following directories:

    - process_tabular_data:

        - source_dir = '../data/raw/L2D/tabular'

        - output_dir_processed = '../data/processed_data/L2D'

        - output_dir_tags = '../data/semantic_tags/L2D'

    - add_data_tags:

        - data_dir = '../data/processed/L2D'

        - tags_dir='../data/semantic_tags/L2D'

In [ ]:
from functions.process_tabular_data_L2D import process_tabular_split

process_tabular_split(0,
                    source_dir='../data/raw/L2D/tabular',
                    output_dir='../data/split/L2D/tabular')

In [ ]:
from functions.process_tabular_data_L2D import process_tabular_data
from functions.process_tags_L2D import add_data_tags

process_tabular_data(0,
                    overwrite=True, process_columns=True, 
                    process_osm=False, process_turning=True,
                    time_sleep=2)

In [ ]:
from functions.process_tags_L2D import add_data_tags

add_data_tags(0)

#### 3. Process Frames

- Set up with depth model with pip install git+https://github.com/apple/ml-depth-pro.git

- Set up rfdetr with pip install --ignore-installed rfdetr

- To process only one episode, use process_frames(ep_num,...)

- To process a range of episodes, use process_frames(min_ep,max_ep,...)

- To process a list of episodes, use process_frames(list,...)

- Additionally, you can change the following directories:

    - input_base_dir = '../data/raw/L2D/frames',
    
    - output_base_dir = '../data/processed_frames/L2D'

- You can also include the following cameras:

    - "observation.images.front_left",
    - "observation.images.left_forward", 
    - "observation.images.right_forward",
    - "observation.images.right_backward",
    - "observation.images.rear",
    - "observation.images.left_backward"

**Note**: Including additional cameras will help with the depth, speed, etc. but CONSIDERABLY increases running time.

In [ ]:
from functions.process_frames_L2D import process_frames

process_frames(0,
               cameras_on=["observation.images.front_left"],
               run_dict={"detection": True,
                         "depth": True,
                         "speed": True,
                         'overwrite': True},
                verbose=True)

In [ ]:
from functions.process_lanes_L2D import lane_processing
summary = lane_processing(0)

#### 4. Generate Graphs (Need to Update for the Addition of Speed)

- To process only one episode, use generate_graphs(ep_num,...)

- To process a range of episodes, use generate_graphs(min_ep,max_ep,...)

- To process a list of episodes, use generate_graphs(list,...)

- Additionally, you can change the following directories:

    - source_data_dir = '../data/processed/L2D',
    
    - processed_frame_dir = '../data/processed_frames/L2D',
                    
    - output_dir = '../data/graphical/L2D'

In [ ]:
from functions.graphs import generate_graphs
generate_graphs(0)

#### Quick Visualization of Graph & Tags

In [7]:
from functions.graphs import combined_graph_viewer

ep_num = 0
with open(f"../data/graphical/l2d/{ep_num}_graph.json", "r") as f:
    graph_data = json.load(f)

cyto = combined_graph_viewer({'graph':graph_data})
display(cyto)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'd…

In [9]:
ep_num = 0
with open(f"../data/semantic_tags/l2d/{ep_num}_graph.json", "r") as f:
    graph_tags = json.load(f)
graph_tags

{'policy': 'STUDENT',
 'instructions': 'Drive straight up to the yield sign and take first exit from the roundabout',
 'action_tag': 'roundabout',
 'traffic_control_tag': 'roundabout',
 'road_feature_tags': ['bike_friendly',
  'pedestrian_area',
  'rural',
  'sidewalk_present'],
 'environment_tags': ['low_visibility_possible',
  'off_peak_hours',
  'winter_conditions_possible']}

In [5]:
from functions.graphs import graph_to_dfs

nodes_df, edges_df = graph_to_dfs(graph_data)
nodes_df.iloc[8].dropna()

id                                                            Veh_FL_A_1
type                                                             vehicle
lane_classification                                     out_of_lane_left
lane_overlap_ratio                                              0.003677
distance_m                                                      3.432257
dist_to_ego                                                     4.675622
speed_ms                                                        9.176929
speed_kmh                                                      33.036946
velocity_ms            [-8.72930463576159, -2.8311258278145695, -4.52...
velocity_kmh           [-31.425496688741724, -10.19205298013245, -16....
Name: 8, dtype: object